<img src = "https://imgur.com/s4wTnl7.jpg" align = "center">

<h1 align=center><font size = 5>CONTENT-BASED FILTERING</font></h1>

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore Content-based recommendation systems and implement a simple version of one using Python and the Pandas library.

### Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
- <a href="#ref1">Acquiring the Data</a><br />
- <a href="#ref2">Preprocessing</a><br />
- <a href="#ref3">Content-Based Filtering</a>
</div>
<br>

<a id="ref1"></a>
# Acquiring the Data

To acquire and extract the data, simply run the following Bash scripts:  
Dataset acquired from [GroupLens](http://grouplens.org/datasets/movielens/). Lets download the dataset. To download the data, we will use **`!wget`**. To download the data, we will use `!wget` to download it from IBM Object Storage.  
__Did you know?__ When it comes to Machine Learning, you will likely be working with large datasets. As a business, where can you host your data? IBM is offering a unique opportunity for businesses, with 10 Tb of IBM Cloud Object Storage: [Sign up now for free](http://cocl.us/ML0101EN-IBM-Offer-CC)

Now you're ready to start working with the data!

<a id="ref2"></a>
# Preprocessing

First, let's get all of the imports out of the way:

In [0]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read each file into their Dataframes:

In [2]:
anime_df = pd.read_csv('https://query.data.world/s/5nvyq67afygsdal4uriv62im6nlpqp')
ratings_df = pd.read_csv('https://query.data.world/s/jdkzynvaig5dd4rbuowrhmvltuwl2e')
anime_df.tail()

,anime_id,name,genre,type,episodes,rating,members
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175
12293,26081,Yasuji no Pornorama: Yacchimae!!,Hentai,Movie,1,5.46,142


Let's also remove the year from the __title__ column by using pandas' replace function and store in a new __year__ column.

In [3]:
anime_df['name'] = anime_df['name'].apply(lambda x: x.strip())
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


With that, let's also split the values in the __Genres__ column into a __list of Genres__ to simplify future use. This can be achieved by applying Python's split string function on the correct column.

In [4]:
#Every genre is separated by a | so we simply have to call the split function on |
anime_df['genre'] = anime_df.genre.str.split(",")
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",TV,64,9.26,793665
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur...",TV,51,9.25,114262
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",TV,24,9.17,673572
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur...",TV,51,9.16,151266


Since keeping genres in a list format isn't optimal for the content-based recommendation system technique, we will use the One Hot Encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature. This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable since genres won't be important for our first recommendation system.

In [16]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
anmeWithGenres_df = anime_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in anime_df.iterrows():
    anmeWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
anmeWithGenres_df = anmeWithGenres_df.fillna(0)
anmeWithGenres_df.head()

,anime_id,name,genre,type,episodes,rating,members,Fantasy
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,9.37,200630,1.0
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",TV,64,9.26,793665,1.0
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur...",TV,51,9.25,114262,1.0
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",TV,24,9.17,673572,1.0
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur...",TV,51,9.16,151266,1.0


Next, let's look at the ratings dataframe.

In [17]:
ratings_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.

<a id="ref3"></a>
# Content-Based recommendation system

Now, let's take a look at how to implement __Content-Based__ or __Item-Item recommendation systems__. This technique attempts to figure out what a user's favourite aspects of an item is, and then recommends items that present those aspects. In our case, we're going to try to figure out the input's favorite genres from the movies and ratings given.

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the __userInput__. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .

In [18]:
userInput = [
            {'name':'Kimi no Na wa.', 'rating':9.37},
            {'name':'Fullmetal Alchemist: Brotherhood', 'rating':9.26},
            {'name':"Ginga Eiyuu Densetsu", 'rating':9.11},
            {'name':"Gintama", 'rating':9.04},
            {'name':"Koe no Katachi", 'rating':9.05}
         ] 
inputAnime = pd.DataFrame(userInput)
inputAnime

,name,rating
0,Kimi no Na wa.,9.37
1,Fullmetal Alchemist: Brotherhood,9.26
2,Ginga Eiyuu Densetsu,9.11
3,Gintama,9.04
4,Koe no Katachi,9.05


#### Add movieId to input user
With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [19]:
#Filtering out the movies by title
inputId = anime_df[anime_df['name'].isin(inputAnime['name'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputAnime = pd.merge(inputId, inputAnime)
#Dropping information we won't use from the input dataframe
inputAnime = inputAnime.drop('genre', 1)
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputAnime

,anime_id,name,type,episodes,rating,members
0,32281,Kimi no Na wa.,Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,TV,64,9.26,793665
2,820,Ginga Eiyuu Densetsu,OVA,110,9.11,80679
3,28851,Koe no Katachi,Movie,1,9.05,102733
4,918,Gintama,TV,201,9.04,336376


We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.

In [20]:
userAnime = anmeWithGenres_df[anmeWithGenres_df['anime_id'].isin(inputAnime['anime_id'].tolist())]
userAnime

,anime_id,name,genre,type,episodes,rating,members,Fantasy
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,9.37,200630,1.0
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",TV,64,9.26,793665,1.0
7,820,Ginga Eiyuu Densetsu,"[Drama, Military, Sci-Fi, Space]",OVA,110,9.11,80679,1.0
11,28851,Koe no Katachi,"[Drama, School, Shounen]",Movie,1,9.05,102733,1.0
12,918,Gintama,"[Action, Comedy, Historical, Parody, Samur...",TV,201,9.04,336376,1.0


We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.

In [21]:
#Resetting the index to avoid future issues
userAnime = userAnime.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userAnime.drop('anime_id', 1).drop('name', 1).drop('genre', 1).drop('type', 1).drop('episodes', 1).drop('members', 1)
userGenreTable

,rating,Fantasy
0,9.37,1.0
1,9.26,1.0
2,9.11,1.0
3,9.05,1.0
4,9.04,1.0


Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling Pandas's "dot" function.

In [22]:
inputAnime['rating']

0    9.37
1    9.26
2    9.11
3    9.05
4    9.04
Name: rating, dtype: float64

In [23]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputAnime['rating'].astype(int))
#The user profile
userProfile

rating     412.47
Fantasy     45.00
dtype: float64

Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.

Let's start by extracting the genre table from the original dataframe:

In [24]:
#Now let's get the genres of every movie in our original dataframe
genreTable = anmeWithGenres_df.set_index(anmeWithGenres_df['anime_id'])
#And drop the unnecessary information
genreTable = genreTable.drop('anime_id', 1).drop('name', 1).drop('genre', 1).drop('members', 1).drop('type', 1).drop('episodes', 1).drop('rating', 1)
genreTable.head()

,Fantasy
anime_id,
32281,1.0
5114,1.0
28977,1.0
9253,1.0
9969,1.0


In [25]:
genreTable.shape

(12294, 1)

With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.

In [26]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

anime_id
32281    0.098367
5114     0.098367
28977    0.098367
9253     0.098367
9969     0.098367
dtype: float64

In [27]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

anime_id
26081    0.098367
6796     0.098367
2089     0.098367
373      0.098367
28789    0.098367
dtype: float64

Now here's the recommendation table!

In [28]:
#The final recommendation table
anime_df.loc[anime_df['anime_id'].isin(recommendationTable_df.head(20).keys())]

,anime_id,name,genre,type,episodes,rating,members
4086,3024,Ribbon no Kishi,"[Action, Fantasy, Mystery, Romance]",TV,52,6.81,1654
4087,5201,Salad Juu Yuushi Tomatoman,"[Adventure, Comedy]",TV,50,6.81,443
4088,23459,Sekai Seifuku: Bouryaku no Zvezda - Shin Zvezd...,"[Action, Comedy, Fantasy]",Special,1,6.81,10141
4089,533,Sensei no Ojikan: Doki Doki School Hours,"[Comedy, Slice of Life]",TV,13,6.81,5137
4090,19391,Shingeki no Kyojin Picture Drama,"[Comedy, Parody]",Special,9,6.81,26363
4091,420,Spriggan,"[Adventure, Military, Sci-Fi, Shounen]",Movie,1,6.81,13256
4092,12857,Star Driver the Movie,"[Action, Mecha, Romance, Shounen]",Movie,1,6.81,7865
4093,10702,Tekken: Blood Vengeance,"[Action, Fantasy, Martial Arts, Sci-Fi, Su...",Movie,1,6.81,12816
4094,2114,Tenkuu Senki Shurato: Sousei e no Anto,"[Action, Fantasy, Magic, Shounen]",OVA,6,6.81,916
4095,7330,Tottoko Hamtarou Movie 2: Ham-Ham Hamuuja! Mab...,"[Adventure, Comedy, Kids]",Movie,1,6.81,1201


### Advantages and Disadvantages of Content-Based Filtering

##### Advantages
* Learns user's preferences
* Highly personalized for the user

##### Disadvantages
* Doesn't take into account what others think of the item, so low quality item recommendations might happen
* Extracting data is not always intuitive
* Determining what characteristics of the item the user dislikes or likes is not always obvious

## Want to learn more?

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler).

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at [Watson Studio](https://cocl.us/ML0101EN_DSX)

### Thanks for completing this lesson!

Notebook created by: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, Gabriel Garcez Barros Sousa

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).​